In [243]:
#Created: 15.02.2016
#Last modified: 18.02.2016
#               29.02.2016 add Corr and Dis cols
#               25.05.2016 updated input file and created truncated file for testing
#               24.05.2016 tested reading in file produced in birdhouse analogs process
#               05.09.2016 format test file for dc chart brush problem

In [1]:
import numpy as np
import pandas as pd
import collections
import os

In [2]:
num_analogues = 20 #number of analogues searched for
num_cols = 3 #dateAnlg, Dis, Corr

In [16]:
#To read file produced in birdhouse
dfS = pd.DataFrame()
#analogs = 'http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/output_txt-0797016c-378e-11e6-91dd-41d8cd554993.txt'
analogs = 'http://localhost:8090/wpsoutputs/flyingpigeon/analogs-c26ba398-711e-11e6-8d5a-fb3204a8dad1.txt'
dfS = pd.read_csv(analogs, delimiter=r"\s+", index_col=0)
dfS.head()

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,...,cor11,cor12,cor13,cor14,cor15,cor16,cor17,cor18,cor19,cor20
date,,,,,,,,,,,,,,,,,,,,,
20130701,20140714,20140713,20140715,20140716,20140704,20140703,20140717,20140712,20140604,20140718,...,0.231885,0.171633,0.427227,-0.161170,0.092983,0.073524,0.134382,0.043228,-0.329172,-0.182452
20130702,20140714,20140715,20140716,20140717,20140713,20140718,20140705,20140605,20140704,20140719,...,-0.143559,0.193200,0.222968,0.537877,-0.237638,0.058815,0.052678,0.100445,0.263816,-0.206209


In [17]:
# Create dataframe from analogues file
# Latest file: /home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2016.dat
# 2015: /home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2015.dat
# Note that last day in 2015 file is 20151225 (and NOT 20151231) because that is the last sunday of Dec 2015.

#dfS = pd.DataFrame()
#dfS = pd.read_csv('/home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2016.dat', delimiter=' ', index_col=0)
#dfS.set_index('Date', inplace=True)

#dfS.head(5)

#dfS.index.name

In [18]:
dfS.tail(5)

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,...,cor11,cor12,cor13,cor14,cor15,cor16,cor17,cor18,cor19,cor20
date,,,,,,,,,,,,,,,,,,,,,
20130701,20140714,20140713,20140715,20140716,20140704,20140703,20140717,20140712,20140604,20140718,...,0.231885,0.171633,0.427227,-0.161170,0.092983,0.073524,0.134382,0.043228,-0.329172,-0.182452
20130702,20140714,20140715,20140716,20140717,20140713,20140718,20140705,20140605,20140704,20140719,...,-0.143559,0.193200,0.222968,0.537877,-0.237638,0.058815,0.052678,0.100445,0.263816,-0.206209


In [19]:
dfS.index

Int64Index([20130701, 20130702], dtype='int64')

In [20]:
len(dfS.index)

2

In [21]:
#Chop off 2016 rows NO!!! there will still be 2016 dates found as analogues, which messes up js script
#dfS = dfS[dfS.index <= 20151231]

In [22]:
#SAVE DATE INDEX
#DF['id2'] = DF.index
dfS['dateRef'] = dfS.index
dfS['dateRef'].head(10)

date
20130701    20130701
20130702    20130702
Name: dateRef, dtype: int64

In [23]:
#TRANSPOSE ANLG DATE COLS INTO SINGLE COL

#Define temporary df containing only anlg dates cols
df_anlg = dfS.iloc[:, 0:20]

#remove index name before transpose (stack)
df_anlg.index.name = ""

#transpose (stack)into single col
dateStack = df_anlg.stack()
#dateStack

In [24]:
#TRANSPOSE DIS COLS INTO SINGLE COL

#Define temporary df containing only dis cols
df_dis = dfS.iloc[:, 20:40]

#remove index name before transpose (stack)
df_dis.index.name = ""

#transpose (stack) into single col and take absolute value
disStack = df_dis.stack().abs()
disStack.head(5)

20130701  dis1    19780.337
          dis2    19940.523
          dis3    19976.523
          dis4    20245.037
          dis5    20309.299
dtype: float64

In [25]:
#TRANSPOSE CORR COLS INTO SINGLE COL

#Define temporary df containing only corr cols
df_corr = dfS.iloc[:, 40:60]

#remove index name before transpose (stack)
df_corr.index.name = ""

#transpose (stack) into single col
corrStack = df_corr.stack()
corrStack.head(5)

20130701  cor1    0.376397
          cor2    0.290715
          cor3    0.399058
          cor4    0.230292
          cor5    0.463296
dtype: float64

In [26]:
#BUILD NEW DF IN CORRECT FORMAT

#CREATE DF OF CORRECT DIMENSIONS (n x num_cols)
df_all = dfS.iloc[:, 0:num_cols]
#rename cols
df_all.columns = ['dateAnlg', 'Dis', 'Corr']

In [27]:
#dfS.index.tolist()
df_all.head(5)

,dateAnlg,Dis,Corr
,,,
20130701,20140714,20140713,20140715
20130702,20140714,20140715,20140716


In [28]:
df_all = df_all.set_index(dfS['dateRef'] )
df_all.head(10)

,dateAnlg,Dis,Corr
dateRef,,,
20130701,20140714,20140713,20140715
20130702,20140714,20140715,20140716


In [29]:
#REPLICATE INDEX COL 20 TIMES PER ROW

#http://stackoverflow.com/questions/26777832/replicating-rows-in-a-pandas-data-frame-by-a-column-value
#df2 = df.loc[np.repeat(df.index.values,df.n)]
df_all = df_all.loc[np.repeat(df_all.index.values,num_analogues)]

In [30]:
#REPLACE COLS WITH STACKED ANLG DATES, DIS AND CORR
df_all.iloc[:,0] = list(dateStack)
df_all.iloc[:,1] = list(disStack)
df_all.iloc[:,2] = list(corrStack)
df_all.head(5)

,dateAnlg,Dis,Corr
dateRef,,,
20130701,20140714,19780.337,0.376397
20130701,20140713,19940.523,0.290715
20130701,20140715,19976.523,0.399058
20130701,20140716,20245.037,0.230292
20130701,20140704,20309.299,0.463296


In [31]:
df_all.shape

(40, 3)

In [32]:
df_all.tail(5)

,dateAnlg,Dis,Corr
dateRef,,,
20130702,20140707,20911.913,0.058815
20130702,20140711,20971.973,0.052678
20130702,20140604,20983.053,0.100445
20130702,20140702,20989.380,0.263816
20130702,20140608,21077.734,-0.206209


In [33]:
dfS_select = df_all[df_all.index <= 19600101]

In [35]:
dfS_select.tail(10)

,dateAnlg,Dis,Corr
dateRef,,,


In [36]:
#SAVE TO CSV FILE

#df_all.to_csv('analogues_19480101_20151225.json', float_format='%.2f', sep='\t')
df_all.to_csv('test2.json', float_format='%.2f', sep='\t')

#df_all.to_csv('analogues_19480101_20160520.json', float_format='%.2f', sep='\t')

#dfS_select.to_csv('analogues_select.json', float_format='%.2f', sep='\t')

In [37]:
pwd

u'/homel/cnangini/PROJECTS/analogues'

In [86]:
print df_all['Corr'].min(), df_all['Corr'].max()

-0.0773818303236 0.952985818969


In [87]:
print df_all['Dis'].min(), df_all['Dis'].max()

284.431297359 1369.41118104


In [31]:
#check 1970-1979
df_check =  df_all[ (df_all['dateAnlg'] >= 19700101) & (df_all['dateAnlg'] <= 19791231) ]


#((df['column1']=='banana') & (df['colour']=='green'))

In [32]:
df_check.shape

(75935, 3)

In [33]:
df_check =  df_all[ (df_all['dateAnlg'] >= 20100101) & (df_all['dateAnlg'] <= 20191231) ]

In [34]:
df_check.shape

(44554, 3)

In [35]:
#cut file for testing purposes
df_select = df_all.iloc[19000:19100,:]

In [36]:
df_select.shape

(100, 3)

In [37]:
df_select.tail(10)

,dateAnlg,Dis,Corr
dateRef,,,
19500812,19840828,449.988840,0.546204
19500812,19590909,453.044659,0.612506
19500812,20030819,453.638210,0.559760
19500812,19970801,457.039066,0.597291
19500812,19980815,459.252587,0.534387
19500812,19990829,466.308358,0.580896
19500812,19510820,466.860763,0.727749
19500812,20130816,467.150673,0.623782
19500812,19990717,471.338490,0.571134


In [38]:
df_chunk = df_all.iloc[149000:149100,:]
df_chunk2 = df_all.iloc[106000:106100,:]

In [39]:
df_select = df_select.append(df_chunk)
df_select = df_select.append(df_chunk2)
#df_select = pd.concat([df_select,df_chunk])

In [40]:
df_select.shape

(300, 3)

In [41]:
#SAVE SELECTION TO TSV FILE

#df_select.to_csv('analogues_reformat_all_select.tsv', float_format='%.2f', sep='\t')

In [42]:
print df_all['Corr'].min(), df_all['Corr'].max()

-0.0773818303236 0.952985818969


In [43]:
print df_all['Dis'].min(), df_all['Dis'].max()

284.431297359 1369.41118104


In [44]:
df_select['Corr'].max()
df_select['Corr'].min()
print df_select['Corr'].min(), df_select['Corr'].max()

0.124791552993 0.821019151951


In [45]:
df_select['Dis'].max()
df_select['Dis'].min()
print df_select['Dis'].min(), df_select['Dis'].max()

350.217929418 696.281040107
